# 나만의 한국어 사전 구축하기: parsing
1. 먼저 다양한 형태소 분석기(꼬꼬마, 트위터, 은전한닢 등)가 들어있는 konlpy를 설치하자

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 170kB/s 
     |████████████████████████████████| 2.4MB 51.2MB/s 
     |████████████████████████████████| 92kB 13.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


2. 이제 어떤 형태소 분석기를 쓸지 결정해야 한다.  
아래의 그림은 문자의 개수에 따른, KoNLPy를 써서 형태소 분석과 품사 태깅 시 소요되는 시간을 뜻하는데  
일단 데이터가 클 수록 꼬꼬마(Kkma)가 성능이 좋으니, 연습삼아 꼬꼬마로 이용해보자
![image](https://user-images.githubusercontent.com/38516906/76173857-8336cb00-61e6-11ea-9d82-5d73615f5fd6.png)


3. 데이터는 [국립국어원 언어정보나눔터 > 통합자료실 > 데이터베이스 자료 > 말뭉치 파일](https://ithub.korean.go.kr/user/total/database/corpusManager.do) > 전화대화_전화통화, 전자전사자료(원시: 7CM00039.txt)  
를 쓸 것이다.  
저작권 때문에 파일을 올릴 수 없으니, 직접 회원가입해서 다운받도록 하자

4. 먼저 파일을 업로드하자.
아래의 코드는 구글 colab에서 진행 시 사용할 수 있는 코드이다.

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 7CM00039.txt to 7CM00039.txt


5. 이제 parsing을 하고 사전을 만들어보자
 * 사용 파싱 라이브러리: BeautifulSoup
 * 파일형식: TEI(Text Encoding Initiative) - 
 텍스트를 디지털 형식으로 표현하기 위한 표준을 공동으로 개발하고 유지하는 컨소시엄

![image](https://user-images.githubusercontent.com/38516906/76177276-052ef000-61f7-11ea-8dd0-16b88b79f7ff.png)


In [39]:
import codecs
from konlpy.tag import Kkma
from bs4 import BeautifulSoup
import copy

def make_myDict(fileName, pass_corpus, del_word_list):
  ##### read file #####
  with open(file = fileName, mode="r", encoding="utf-16") as f: # utf-16으로 안하면 오류남
    tei = f.read()
    soup = BeautifulSoup(tei, "html.parser")
    mylist = soup.select("s") # s 태그 글자 모두 추출
    result = str([text.getText() for text in mylist]) # 리스트 값 모음으로 변경
  
    # make dictionary #
    word_dict = {}
    '''
    (꼬꼬마는 태그가 너무 많아서)임의로 필요없다고 생각하는 부분(콤마, 감탄사 등)은
    코퍼스 앞글자만(UN이면 U만) 따와서 임의로 제거
    '''

    lines = result.split("\n")
    for line in lines:
      malist = kkma.pos(line)
      for word in malist:
        if word[1][0] != 'I' and word[1][0] != 'S' and word[1][0] != 'U': # 임의제거
          if not word[0] in word_dict:
            word_dict[word[0]] = 0
          word_dict[word[0]] += 1


  ##### delete optional word #####
  modify_word_dict = copy.deepcopy(word_dict)
  for del_word in del_word_list:
    modify_word_dict.pop(del_word, None)


  ##### print result #####
  keys1 = sorted(word_dict.items(), key=lambda x:x[1], reverse=True) #빈도가 높은 순으로 정렬
  print("[origin result]")
  for word, count in keys1:
    print("{}({})".format(word, count), end="")

  keys2 = sorted(modify_word_dict.items(), key=lambda x:x[1], reverse=True) #빈도가 높은 순으로 정렬
  print("\n\n[modify result(delete wanted words)]")
  for word, count in keys2: #[:50]:
    print("{}({})".format(word, count), end="")











if __name__ == "__main__":
  kkma = Kkma()

  fileName = "7CM00039.txt"
  del_word_list = ["XX", "있", "는데", "잖아", "1", '치', 'X', '에서', '외', '머', '막'] # 임의설정
  pass_corpus = ['I', 'S', 'U'] #### 수정 필요

  make_myDict(fileName, pass_corpus, del_word_list)

[origin result]
어(67)아(35)하(18)이(13)고(12)야(8)우(7)ㄴ(7)는(7)나(7)었(7)이십(7)왜(7)와(6)여(6)분(6)뭐(5)것(5)오(5)언니(5)에(5)알(5)모르(4)ㄴ가(4)안(4)가(4)지(4)내일(4)주(4)대고(3)그렇(3)보(3)우리(3)녹음(3)되(3)니(3)말(3)도(3)다(3)있(3)는데(3)문(3)아이(3)라(3)그러(3)거(3)오빠(3)면(3)마시(3)말하(2)이거(2)데(2)무슨(2)이래(2)바(2)정말(2)네(2)이나(2)잖아(2)더니(2)때리(2)후(2)계속(2)미안(2)닫(2)끄(2)았(2)동안(2)좋(2)보세(2)게(2)그럼(2)러(2)어떡하(2)받(2)그리하(2)놓(2)추우(2)요(2)또(2)술(2)냐(2)못(2)집(2)XX(2)한(2)아르바이트(2)시(2)매일(2)얼마나(2)엄청(2)뭔(1)이상(1)기계(1)언어(1)경로(1)개발(1)연구원(1)계(1)발(1)ㄴ데(1)지영(1)예쁘(1)빨리(1)그냥(1)지금(1)의(1)아름다움(1)대하(1)얘기(1)아푸(1)바람(1)붇(1)마이크(1)에다(1)따(1)X(1)눌르(1)얼만큼(1)불(1)구(1)1(1)맞(1)하래(1)몰(1)은(1)아니(1)연구(1)에서(1)본데(1)외(1)이렇(1)굳이(1)어도(1)으니까(1)입(1)아프(1)머(1)막(1)구경(1)부(1)마(1)싸(1)아야(1)노(1)않(1)아요(1)아우(1)열(1)수업(1)장난(1)드(1)쉬(1)던데(1)어띃게(1)밥(1)을(1)먹(1)빵(1)ㄹ(1)이것(1)어느(1)지애(1)너(1)저기(1)다가(1)거기(1)저거(1)내(1)돈(1)얼마(1)아르(1)바이(1)저(1)유진(1)기로(1)바로(1)차(1)양(1)다음(1)부터(1)일주일(1)육(1)일(1)까지(1)여도(1)터(1)중요(1)속(1)은데(1)ㄹ라고(1)유(1)겠(1)선(1)더(1)치(1)

[modify result(delete wanted words)]
어(67)아(35)하(18)이(13)고(12)야(8)우(7)ㄴ(7)는(7)나(7)

# Reference
* TEI 설명: https://tei-c.org/tools/
* parsing 코드: https://www.crummy.com/software/BeautifulSoup/bs4/doc/